In [134]:
import torch 

# 8 bits quantization

f = torch.rand(10,20)*100-50
nbits = 4

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weight_max = torch.max(f)
fixed_exp = torch.ceil(torch.log2(weight_max/(2**(nbits-1)-1))) + 1
print(fixed_exp)

def float2bin(f, fixed_exp, nbits=8):
    s = torch.sign(f)
    f = f * s
    s = (s * (-1) + 1.) * 0.5
    s= s.unsqueeze(-1)
    f = f/(2**fixed_exp)
    m = integer2bit(f - f % 1,num_bits = nbits-1)
    dtype = f.type()
    out = torch.cat([s, m], dim=-1).type(dtype)
    return out

def integer2bit(integer, num_bits=7):
    dtype = integer.type()
    exponent_bits = -torch.arange(-(num_bits - 1), 1).type(dtype)
    exponent_bits = exponent_bits.repeat(integer.shape + (1,))
    out = integer.unsqueeze(-1) / 2 ** exponent_bits
    return (out - (out % 1)) % 2

def bin2float(b, fixed_exp, nbits=8):
    dtype = torch.float32
    s = torch.index_select(b, -1, torch.arange(0, 1).to(device))
    m = torch.index_select(b, -1, torch.arange(1,nbits).to(device))
    out = ((-1) ** s).squeeze(-1).type(dtype)
    exponents = -torch.arange(-(nbits - 2.), 1.).to(device)
    exponents = exponents.repeat(b.shape[:-1] + (1,))
    e_decimal = torch.sum(m * 2 ** (exponents), dim=-1)
    out *= e_decimal * 2 ** fixed_exp
    return out


tensor(4.)


In [135]:
print(f)
bin = float2bin(f,fixed_exp,nbits).to(torch.int8)
after = bin2float(bin,fixed_exp,nbits)
print(after)
after_after = float2bin(after,fixed_exp,nbits).to(torch.int8)
print(after_after)
bin2float(after_after,fixed_exp,nbits)

tensor([[ 4.7771e+01, -3.4227e+01, -9.5253e-03,  1.0352e+01,  1.7059e+01,
         -3.5377e+01, -3.5268e+01,  4.0724e+01,  3.1617e+00, -4.2631e+00,
          1.9126e+01,  3.6907e+01,  1.7489e+01,  3.7150e+01, -1.4453e+01,
          4.4430e+01,  2.9868e+01,  1.5259e+01, -1.7054e+01, -3.1845e-02],
        [-1.1995e+01, -4.6654e+01, -4.9111e+01, -3.6493e+01, -4.2246e+00,
         -4.6009e+01, -3.7933e+01, -3.1223e+00,  1.2779e+01, -1.5551e+00,
          2.2232e+00,  3.3257e+01,  1.8520e+01, -1.9330e+01, -2.3617e+01,
         -3.1402e+01, -3.8065e+01, -2.1549e+01,  1.1580e+01,  1.6794e+01],
        [-1.9352e+00, -4.6217e-01,  3.2686e+01,  2.1702e+01,  4.1500e-01,
         -1.6708e+01,  2.3037e+01, -1.9879e+01, -3.2784e+01,  2.2521e+01,
         -3.9120e+01, -1.3572e+00, -3.5718e+01,  4.3591e-01,  3.4373e+01,
          2.9865e+01,  4.4390e+01,  1.9883e+00,  3.7110e+01, -2.2099e+01],
        [ 3.4160e+01,  2.1451e+01, -1.6273e+01,  3.7362e+01,  5.1774e+00,
          1.4325e+01, -4.7082e+01, 

tensor([[ 32., -32.,   0.,   0.,  16., -32., -32.,  32.,   0.,   0.,  16.,  32.,
          16.,  32.,   0.,  32.,  16.,   0., -16.,   0.],
        [  0., -32., -48., -32.,   0., -32., -32.,   0.,   0.,   0.,   0.,  32.,
          16., -16., -16., -16., -32., -16.,   0.,  16.],
        [  0.,   0.,  32.,  16.,   0., -16.,  16., -16., -32.,  16., -32.,   0.,
         -32.,   0.,  32.,  16.,  32.,   0.,  32., -16.],
        [ 32.,  16., -16.,  32.,   0.,   0., -32.,  32., -32., -16., -16.,   0.,
           0., -32.,   0.,   0.,   0.,  16.,   0.,   0.],
        [-32., -48.,   0.,  32., -16.,   0.,   0.,   0., -16., -16.,  48., -32.,
           0.,   0.,   0.,   0.,   0., -16.,   0., -32.],
        [-32.,   0.,  16.,  16.,   0.,   0.,  16.,   0.,  32.,  32.,  16., -16.,
          32.,   0.,   0.,   0., -32.,  16.,   0.,   0.],
        [ 32.,  16.,  16., -16.,   0.,   0.,  16., -16.,   0.,   0.,  16.,  32.,
           0.,  16.,   0., -16., -32.,  16.,  32.,  32.],
        [ 32.,   0., -32., 